# Projekt zaliczeniowy - Jacek Kwiendacz - Katowice - grupa 1

## Wybrany temat: temat 2 - przewidywanie wzięcia pożyczki

CEL:
budowa modelu klasyfikacji binarnej do przewidzenia czy pożyczka zostanie udzielona danemu klientowi.
Dane zostały pobrane z lokalizacji podanej w treści zadania. 

W normalnych warunkach opisałbym dane udostępnione do pracy, zakładam jednak, że sprawdzający będzie zaznajomiony ze specyfiką zadania. 

## Część 1 - przygotowanie danych do modelowania

#### Kolejne kroki:
- pobranie danych do DataFrame
- zmiana formatów pól (daty, pola numeryczne, tekst)
- wykorzystanie zmiennej "city" do pobrania lokalizacji GPS
- wyznaczenie wieku wnioskującego klienta na moment wnioskowania w oparciu o datę złożenia aplikacji oraz o datę urodzenia
- analiza zmiennych numerycznych
- analiza zmiennych kategorycznych
- eksport wyczyszczonych danych do csv

In [1]:
# import pakietów niezbędnych do dalszej pracy

import numpy as np
import pandas as pd
import urllib
from IPython.display import HTML
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import re
from IPython.display import HTML
import statistics as stat
from random import sample

In [2]:
#import danych z pliku csv do DataFrame

col_names = ['ID', 'gender', 'city', 'income', 'birth_date', 'application_date', 'requested_amount', 
             'requested_period', 'financial_obligations', 'employer_name', 'account_bank', 'mobile_verification_flag',
             'var_5', 'var_1', 'offer_amount', 'offer_period', 'interest_rate', 'fee', 'offer_monthly_obligation',
             'filled_form_flag', 'device', 'var_2', 'source', 'var_4', 'logged_in_flag', 'disbursed_flag']
dataset = pd.read_csv('dataset.csv', delimiter = ',', dtype='str', header = None, names = col_names, engine='python', 
                      index_col = False, skiprows = 1 )

# zgodnie z warunkiem zadania - pomijam zmienną 'logged_in_flag'

dataset = dataset.drop(columns = ['logged_in_flag'])
HTML(dataset.head().to_html())

,ID,gender,city,income,birth_date,application_date,requested_amount,requested_period,financial_obligations,employer_name,account_bank,mobile_verification_flag,var_5,var_1,offer_amount,offer_period,interest_rate,fee,offer_monthly_obligation,filled_form_flag,device,var_2,source,var_4,disbursed_flag
0,ID000002C20,Female,Delhi,20000,23-May-78,15-May-15,300000,5,0,CYBOSOL,HDFC Bank,N,0,HBXX,nan,nan,nan,nan,nan,N,Web-browser,G,S122,1,0
1,ID002616Q10,Female,Madurai,11000,10-Jul-89,03-May-15,0,0,0,SAI BPO SERVICES LTD,ICICI Bank,N,0,HBXX,nan,nan,nan,nan,nan,N,Mobile,C,S133,1,0
2,ID000004E40,Male,Mumbai,35000,07-Oct-85,04-May-15,200000,2,0,TATA CONSULTANCY SERVICES LTD (TCS),ICICI Bank,Y,13,HBXA,200000,2,13.25,nan,6762.9,N,Web-browser,G,S122,3,0
3,ID000007H20,Male,Panchkula,22500,10-Oct-81,19-May-15,600000,4,0,ALCHEMIST HOSPITALS LTD,State Bank of India,Y,0,HBXX,450000,4,nan,nan,nan,N,Web-browser,B,S143,1,0
4,ID000008I30,Male,Saharsa,35000,30-Nov-87,09-May-15,1000000,5,0,BIHAR GOVERNMENT,State Bank of India,Y,10,HBXX,920000,5,nan,nan,nan,N,Web-browser,B,S143,3,0


In [3]:
# zmiana formatu pól datowych: zmieniam format 'DD-MMM-YY' na format 'YYYY-MM-DD'

months_dict = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06',
               'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}

def date_transform(date:str, twentieth_cent=False):
    day = date[:2]
    month = months_dict[date[3:6]]
    if twentieth_cent:
        return '19' + date[-2:] + "-" + month + "-" + day
    else:
        return '20' + date[-2:] + "-" + month + "-" + day

dataset['birth_date'] = dataset.apply(lambda x: date_transform(x['birth_date'], True), axis=1)
dataset['application_date'] = dataset.apply(lambda x: date_transform(x['application_date']), axis=1)

HTML(dataset.head().to_html())

,ID,gender,city,income,birth_date,application_date,requested_amount,requested_period,financial_obligations,employer_name,account_bank,mobile_verification_flag,var_5,var_1,offer_amount,offer_period,interest_rate,fee,offer_monthly_obligation,filled_form_flag,device,var_2,source,var_4,disbursed_flag
0,ID000002C20,Female,Delhi,20000,1978-05-23,2015-05-15,300000,5,0,CYBOSOL,HDFC Bank,N,0,HBXX,nan,nan,nan,nan,nan,N,Web-browser,G,S122,1,0
1,ID002616Q10,Female,Madurai,11000,1989-07-10,2015-05-03,0,0,0,SAI BPO SERVICES LTD,ICICI Bank,N,0,HBXX,nan,nan,nan,nan,nan,N,Mobile,C,S133,1,0
2,ID000004E40,Male,Mumbai,35000,1985-10-07,2015-05-04,200000,2,0,TATA CONSULTANCY SERVICES LTD (TCS),ICICI Bank,Y,13,HBXA,200000,2,13.25,nan,6762.9,N,Web-browser,G,S122,3,0
3,ID000007H20,Male,Panchkula,22500,1981-10-10,2015-05-19,600000,4,0,ALCHEMIST HOSPITALS LTD,State Bank of India,Y,0,HBXX,450000,4,nan,nan,nan,N,Web-browser,B,S143,1,0
4,ID000008I30,Male,Saharsa,35000,1987-11-30,2015-05-09,1000000,5,0,BIHAR GOVERNMENT,State Bank of India,Y,10,HBXX,920000,5,nan,nan,nan,N,Web-browser,B,S143,3,0


### Wykorzystanie zmiennej "city":

Przegląd danych pozwala stwierdzić, że zmienna "city" jets informacją mocno specyficzną dla danego rekordu danych. Trudno tę zmienną uznać jako zmienną kategoryczną. Warto jednak wykorzystać informację o lokalizacji złożenia aplikacji.

Wykorzystując narzędzia pozwalające przeszukiwać Internet w poszukiwaniu danych można dla danej miejscowości odnaleźć zmienną charakteryzującą lokalizację w sposób jednolity dla wszystkich miejscowości - np. współrzędne geograficzne.

Poniżej skrypt realizujący ten pomysł:

- przeszukuję wyniki zwracane przez Google,
- przeszukuję wyniki zwracane przez Wikipedię,
- przeszukuję wyniki zwracane przez stronę 'latitude.to'

Na każdej z wywołanych stron wyszukuję informacji o współrzędnych geograficznych.

Do datasetu przypisuję informację o długości i szerokości geograficznej znalezionej dla danej miejscowości (zmienne numeryczne "latitude", "longitude").

In [4]:
# zmienna "city" - pobranie lokalizacji GPS z Internetu (Google, Wikipedia, latitude.to)

# granice długości i szerokości geograficznej dla Indii
min_latitude = 7
max_latitude = 38
min_longitude = 67
max_longitude = 98

# parser - przekształca string na współrzędne wyrażone w stopniach
def get_coords_from_pattern(string, pattern):
    coords = string.split(" ")
    degs = float(coords[0].split(pattern[6])[0])
    mins = float(coords[1].split(pattern[16])[0])
    secs = float(coords[2].split(pattern[33])[0])
    return degs + mins/60.0 + secs/3600.0


# funkcja przekształcająca listę znalezionych łańcuchów tekstowych w listę współrzędnych
def convert_list_to_coord(found, pattern):
    n, e = None, None
    found_n, found_e = [], []
    # wiki pattern
    if 'nsew' not in pattern:
        found_n = [float(item.split(" ")[0].split('°')[0]) for item in found]
        found_e = [float(item.split(" ")[1].split('°')[0]) for item in found]
    # google patterns
    else:
        found_n = [get_coords_from_pattern(item, pattern) for item in found if item[-1]=='n']
        found_e = [get_coords_from_pattern(item, pattern) for item in found if item[-1]=='e']
        found_n = [coord for coord in found_n if coord >= min_latitude and coord <= max_latitude]
        found_e = [coord for coord in found_e if coord >= min_longitude and coord <= max_longitude]
    if found_n:
        n = stat.mean(found_n)
    if found_e:
        e = stat.mean(found_e)               
    return [n, e]


# funkcja pobierająca dane z Wikipedii dla miejscowości zadanej jako argument
def get_wikipedia_data(city):
    pattern = "[0-9]+.[0-9]+°n [0-9]+.[0-9]+°e"
    found = []
    try:
        with urlopen("https://en.wikipedia.org/wiki/" + city) as page:
            soup = BeautifulSoup(page.read(), "html.parser")
            response_list = soup.select(".geo-dec")            
            for item in response_list:
                found += re.findall(pattern, item.get_text().lower())
    except urllib.request.HTTPError:
        pass
    return convert_list_to_coord(found, pattern)


# funkcja pobierająca dane z wyników zapytania w Google dla miejscowości zadanej jako argument  
def get_google_search_data(city):
    pattern = "[0-9]+°[ ][0-9]+'[ ][0-9]+.[0-9]+''[ ][nsew]"
    found = []        
    driver.get('https://www.google.pl/search?q=' + city + '+latitude+and+longitude+in+decimal')
    found = re.findall(pattern, driver.page_source.lower())
    return convert_list_to_coord(found, pattern)


# funkcja pobierająca dane ze strony 'latitude.to' dla miejscowości zadanej jako argument  
def get_latitude_to_data(city):
    pattern = "[0-9]+°[ ][0-9]+'[ ][0-9]+.[0-9]+\"[ ][nsew]"
    found = []        
    driver.get('https://www.google.pl/search?q=' + city + '+india+gps+coords')
    try:
        links = driver.find_elements_by_css_selector('.rc')
        links = [item for item in links if 'latitude.to' in item.text.lower()]
        if links:
            links[0].find_element_by_css_selector('h3').click()
            undone = True
            while undone:
                if 'google' not in driver.current_url:
                    try:
                        found = re.findall(pattern, driver.page_source.lower())
                    except NoSuchElementException:
                        pass                        
                    undone = False
    except NoSuchElementException:
        pass
    return convert_list_to_coord(found, pattern)

In [5]:
# uruchomienie pobierania danych z Internetu -- zakomentowane z uwagi na kilkunastominotuwy czas wykonania
# dane są ściągane na dysk (zapis do .csv), więc krok można pominąć i przeczytać dane z dysku

# cities = sorted(list(set(dataset['city'])))
# driver = webdriver.Chrome('chromedriver')

# coords = {}
# for item in cities:
#     coords[item] = get_wikipedia_data(item)
#     if coords[item][0] == None or coords[item][1] == None:
#         coords[item] = get_google_search_data(item)
#     if coords[item][0] == None or coords[item][1] == None:
#         coords[item] = get_latitude_to_data(item)   
# driver.close()

# cities_coords = pd.DataFrame.from_dict(coords, orient='index', columns = ['city', 'coords'])
# cities_coords.to_csv('C:/Users/Jacek/Desktop/projekt zaliczeniowy/gps_coords.csv')

cities_coords = pd.read_csv('gps_coords.csv', names = ['city', 'latitude', 'longitude'], skiprows = 1)
cities_coords = cities_coords[~pd.isnull(cities_coords['latitude'])]
cities_coords = cities_coords[~pd.isnull(cities_coords['longitude'])]
dataset = pd.merge(dataset, cities_coords, on = 'city', how='left')

### Wykorzystanie zmiennych 'birth_date' i 'application_date'

Zmienne datowe nie niosą informacji istotnej dla wykonywanej analizy.
Można natomiast przekształcić je do zmiennych charakteryzujących np. kredytobiorcę i potencjalnie skorelowanych ze zmienną objaśnianą.

W tym celu przekształcam daty ze zbioru danych i wyliczam wiek kredytobiorcy w latach, zakładam bowiem, że wiek kredytobiorcy może pośrednio wpłynąć na możliwość pozyskania kredytu.

In [6]:
# wyliczenie wieku na moment wnioskowania 

def get_date_diff_in_years(date_start, date_end):
    out  = int(date_end[:4]) - int(date_start[:4])
    out += (int(date_end[5:7]) - int(date_start[5:7]))/12.0
    out += (int(date_end[8:]) - int(date_start[8:]))/365.25
    return round(out, 3)

dataset['age'] = dataset.apply(lambda x: get_date_diff_in_years(x['birth_date'], x['application_date']), axis=1)
HTML(dataset.head(3).to_html())

,ID,gender,city,income,birth_date,application_date,requested_amount,requested_period,financial_obligations,employer_name,account_bank,mobile_verification_flag,var_5,var_1,offer_amount,offer_period,interest_rate,fee,offer_monthly_obligation,filled_form_flag,device,var_2,source,var_4,disbursed_flag,latitude,longitude,age
0,ID000002C20,Female,Delhi,20000,1978-05-23,2015-05-15,300000,5,0,CYBOSOL,HDFC Bank,N,0,HBXX,nan,nan,nan,nan,nan,N,Web-browser,G,S122,1,0,28.610000,77.23000,36.978
1,ID002616Q10,Female,Madurai,11000,1989-07-10,2015-05-03,0,0,0,SAI BPO SERVICES LTD,ICICI Bank,N,0,HBXX,nan,nan,nan,nan,nan,N,Mobile,C,S133,1,0,9.912292,78.10986,25.814
2,ID000004E40,Male,Mumbai,35000,1985-10-07,2015-05-04,200000,2,0,TATA CONSULTANCY SERVICES LTD (TCS),ICICI Bank,Y,13,HBXA,200000,2,13.25,nan,6762.9,N,Web-browser,G,S122,3,0,18.975000,72.82583,29.575


## Analiza zmiennych numerycznych

W analizowanym zbiorze danych identyfikuję 12 zmiennych numerycznych, w tym 3 wyliczone w oparciu i inne dane z datasetu.
Są to:

- income - dochód,
- requested_amount - kwota wnioskowana,
- requested_period - wnioskowany okres kredytowania,
- financial_obligations - obciążenia finansowe klienta,
- offer_amount - kwota oferty,
- offer_period - okres kredytowania oferty,
- interest_rate - stopa procentowa oferty,
- fee - prowizja za udzielenie kredytu,
- offer_monthly_obligation - wysokość raty miesięcznej oferty,
- latitude, longitude - długość i szerokość geograficzna miejsca złożenia wniosku wyrażone w stopniach,
- age - wiek klienta w latach w momencie złożenia wniosku.

Zmienne pochodzące z wejściowego zbioru danych to zmienne kwotowe charakteryzujące możliwości finansowe klienta, parametry oferty wnioskowanej oraz przedstawionej klientowi. Wartości kwotowe wyrażone są w rupiach indyjskich.

Zmienną objaśnianą jest 'disbursed_flag' - zmienna 0/1 oznaczająca fakt wzięcia pożyczki.

Analiza tych zmiennych polegała na:

- wyznaczeniu i przeglądzie statystyk opisowych (w tym percentyle 1%, 5%, 95%, 99%)
- identyfikacji i modyfikacji wartości odstających
- sprawdzenia poziomu braków danych
- uzupełnieniu braków danych

In [7]:
# ustalenie listy zmiennych numerycznych, dostosowanie formatu danych

numeric = ['income', 'requested_amount', 'requested_period', 'financial_obligations', 'offer_amount', 'offer_period', 
           'interest_rate', 'fee', 'offer_monthly_obligation', 'latitude', 'longitude', 'age']

explained = 'disbursed_flag'

for variable in numeric:
    dataset[variable] = dataset[variable].astype(float)

dataset[explained] = dataset[explained].astype(float)

In [8]:
# statystyka opisowa dla zmiennych numerycznych

numeric_features_stats = dataset[numeric].describe(percentiles=[.01, .05, .25, .5, .75, .95, .99]).transpose()
numeric_features_stats = numeric_features_stats.rename(columns={'50%':'median'})
columns_formats = {'count':'{:10.0f}', 'mean':'{:10.2f}', 'std':'{:10.2f}', 'min':'{:10.2f}', '1%':'{:10.2f}', '5%':'{:10.2f}',
                   '25%':'{:10.2f}', 'median':'{:10.2f}', '75%':'{:10.2f}', '95%':'{:10.2f}', '99%':'{:10.2f}', 'max':'{:10.2f}'}
numeric_features_stats.style.format(columns_formats)

,count,mean,std,min,1%,5%,25%,median,75%,95%,99%,max
income,87020,58849.97,2177511.36,0.00,2583.00,10000.00,16500.00,25000.00,40000.00,95000.00,250000.00,444554443.00
requested_amount,86949,230250.70,354206.76,0.00,0.00,0.00,0.00,100000.00,300000.00,1000000.00,1500000.00,10000000.00
requested_period,86949,2.13,2.01,0.00,0.00,0.00,0.00,2.00,4.00,5.00,5.00,10.00
financial_obligations,86949,3696.23,39810.21,0.00,0.00,0.00,0.00,0.00,3500.00,18000.00,40552.00,10000000.00
offer_amount,52407,395010.59,308248.14,50000.00,60000.00,100000.00,200000.00,300000.00,500000.00,1000000.00,1500000.00,3000000.00
offer_period,52407,3.89,1.17,1.00,1.00,2.00,3.00,4.00,5.00,5.00,5.00,6.00
interest_rate,27726,19.20,5.83,11.99,12.99,13.50,15.25,18.00,20.00,31.50,37.00,37.00
fee,27420,5131.15,4725.84,200.00,700.00,1000.00,2000.00,4000.00,6250.00,14000.00,24000.00,50000.00
offer_monthly_obligation,27726,10999.53,7512.32,1176.41,2405.32,3447.10,6491.60,9392.97,12919.04,25444.41,40614.48,144748.28
latitude,85917,20.14,6.04,8.08,9.91,12.97,13.08,18.98,26.71,28.61,30.75,34.55


Kilka obserwacji z analizy miar statystycznych per zmienna:

- income - wyraźna obecność wartości odstających, duże odchylenie standardowe, znaczna rozbieżność między średnią i medianą,
- requested_amount - również spory rostrzał wartości, choć nie tak ewidentny jak w przypadku zmiennej 'income'
- financial_obligations - zerowa mediana wskazuje na przeważającą część populacji z wartością 0

Braki danych:

Łączna liczba obserwacji to 87020.
Jedyne zmienne bez braków danych to 'income' oraz 'age'.

Zmienne charakteryzujące ofertę mają dość niski poziom wypełnienia, szczególnie zmienne 'interest_rate' oraz 'fee'.
Użyteczność tych zmiennych w dalszej analizie może być wątpliwa.

## Identyfikacja i modyfikacja wartości odstających

Jako 'outliery' arbitralnie przyjąłem wartości poza przedziałem (percentyl 1%, percentyl 99%).
Dla tych wartości zastosowałem ograniczenie wartością danego percentyla.
Modyfikacji poddałem zmienne finansowe pochodzące z wejściowego zbioru danych.
Nie modyfikowałem zmiennych: 'age', 'latitude', 'longitude'.

Zastosowanie podmiany zmieniło parametry statystyki opisowej:
- znaczący spadek std_dev dla zmiennej 'income'
- znaczący spadek std_dev dla zmiennej 'financial_obligations'

In [9]:
# identyfikacja i usuwanie outlierów
for feature in list(numeric_features_stats.index)[:9]:
    replacement_min = numeric_features_stats.loc[feature]['1%']
    replacement_max = numeric_features_stats.loc[feature]['99%']
    dataset[feature] = dataset.apply(lambda x: replacement_max if x[feature] > replacement_max else x[feature], axis=1)
    dataset[feature] = dataset.apply(lambda x: replacement_min if x[feature] < replacement_min else x[feature], axis=1)

print("\nStatystyka opisowa zmiennych numerycznych po modyfikacji wartości odstających: \n")
numeric_features_stats = dataset[numeric].describe(percentiles=[.01, .05, .25, .5, .75, .95, .99]).transpose().rename(columns={'50%':'median'})
numeric_features_stats.style.format(columns_formats)


Statystyka opisowa zmiennych numerycznych po modyfikacji wartości odstających: 



,count,mean,std,min,1%,5%,25%,median,75%,95%,99%,max
income,87020,34876.94,35569.25,2583.00,2583.00,10000.00,16500.00,25000.00,40000.00,95000.00,250000.00,250000.00
requested_amount,86949,221673.27,287739.99,0.00,0.00,0.00,0.00,100000.00,300000.00,1000000.00,1500000.00,1500000.00
requested_period,86949,2.13,2.01,0.00,0.00,0.00,0.00,2.00,4.00,5.00,5.00,5.00
financial_obligations,86949,3261.21,7194.89,0.00,0.00,0.00,0.00,0.00,3500.00,18000.00,40527.04,40552.00
offer_amount,52407,392356.17,294815.59,60000.00,60000.00,100000.00,200000.00,300000.00,500000.00,1000000.00,1500000.00,1500000.00
offer_period,52407,3.89,1.17,1.00,1.00,2.00,3.00,4.00,5.00,5.00,5.00,5.00
interest_rate,27726,19.20,5.83,12.99,12.99,13.50,15.25,18.00,20.00,31.50,37.00,37.00
fee,27420,5065.60,4369.50,700.00,700.00,1000.00,2000.00,4000.00,6250.00,14000.00,24000.00,24000.00
offer_monthly_obligation,27726,10928.35,7064.07,2405.32,2405.32,3447.10,6491.60,9392.97,12919.04,25444.41,40614.48,40614.48
latitude,85917,20.14,6.04,8.08,9.91,12.97,13.08,18.98,26.71,28.61,30.75,34.55


## Analiza braków danych

Braki danych to istotny problem w zestawie zmiennych finansowych.
Podobne ilości rekordów z brakami danych per zmienna sugerują ich strukturalny (procesowy) charakter.
W celu potwierdzenia tej hipotezy sprawdziłem wzajemną zależność 'nulli' dla 5 zmiennych, dla których problem jest istotny:

- 'offer_amount', 
- 'offer_period', 
- 'interest_rate', 
- 'fee', 
- 'offer_monthly_obligation'.

Poziom braków dla pozostałych zmiennych jest na tyle mały, że sposób ich uzupełnienia nie powinien istotnie wpłynąć na rozkład zmiennych, ani na wynik dalszych analiz i modelowania.

In [10]:
#null values - struktura
massive_nulls = dataset[['offer_amount', 'offer_period', 'interest_rate', 'fee', 'offer_monthly_obligation']]

nulls = massive_nulls[np.isnan(massive_nulls['offer_amount'])]

for feature in massive_nulls.columns:
    nulls = massive_nulls[np.isnan(massive_nulls[feature])]
    nulls = nulls.describe().transpose()
    nulls = nulls[['count']]
    print("\n" + feature + " is null --> other features statistics:\n")
    display(HTML(nulls.to_html()))


offer_amount is null --> other features statistics:



,count
offer_amount,0.0
offer_period,0.0
interest_rate,0.0
fee,0.0
offer_monthly_obligation,0.0



offer_period is null --> other features statistics:



,count
offer_amount,0.0
offer_period,0.0
interest_rate,0.0
fee,0.0
offer_monthly_obligation,0.0



interest_rate is null --> other features statistics:



,count
offer_amount,24681.0
offer_period,24681.0
interest_rate,0.0
fee,0.0
offer_monthly_obligation,0.0



fee is null --> other features statistics:



,count
offer_amount,24987.0
offer_period,24987.0
interest_rate,306.0
fee,0.0
offer_monthly_obligation,306.0



offer_monthly_obligation is null --> other features statistics:



,count
offer_amount,24681.0
offer_period,24681.0
interest_rate,0.0
fee,0.0
offer_monthly_obligation,0.0


Powyższe wyniki potwierdzają hipotezę o systemowym charakterze braków:
    
- brak informacji o kwocie oferty ('ofer_amount') implikuje braki danych dla pozostałych 4 zmiennych
- brak informacji o okresie oferty (offer_period') występuje wyłącznie w parze z brakiem informacji o 'offer_amount'
- brak informacji o pozostałych trzech parametrach występuje (prawie) zawsze razem, częściej niż brak informacji o kwocie i okresie oferty
- dodatkowo dla niewielkiej ilości rekordów występuje brak informacji o opłacie (zmienna 'fee') mimo informacji o pozostąłych parametrach


## Sposoby uzupełnienia braków danych

Brak szczegółowej wiedzy o procesie kredytowym, z którego te dane pochodzą, uniemożliwia jednoznaczną interpretację braku danych. Wobec tego trudno wybrać właściwą metodę uzupełnienia braków.

Postanowiłem uzupełnić braki danych na kilka sposobów:

1. Dla danych z niewielką ilością braków przyjąłem uzupełnienie medianą.
2. Dla 5 zmiennych ze znaczną ilością braków:

    - uzupełniam braki medianą
    - uzupełniam braki losując wartość z rozkładu normalnego o średniej właściwej dla danej zmiennej oraz o odch. standardowym stanowiącym 1/3 odch. standardowego danej zmiennej (z ograniczeniem do wartości [średnia - odch_stand, średnia + odch_stand])
    - uzupełniam braki danych przepisując je z losowo wybranych rekordów ze zbioru danych z uzupełnionymi wartościami

In [11]:
# uzupełnienie nulli - sposób inputacji

null_values_inputation_median = {
    'income':'median', 
    'requested_amount':'median', 
    'requested_period':'median', 
    'financial_obligations':'median', 
    'offer_amount':'median', 
    'offer_period':'median',
    'interest_rate':'median', 
    'fee':'median', 
    'offer_monthly_obligation':'median',
    'latitude':'median',
    'longitude':'median'
}

null_values_inputation_distribution = {
    'income':'median', 
    'requested_amount':'median', 
    'requested_period':'median', 
    'financial_obligations':'median', 
    'offer_amount':'distribution', 
    'offer_period':'distribution',
    'interest_rate':'distribution', 
    'fee':'distribution', 
    'offer_monthly_obligation':'distribution',
    'latitude':'median',
    'longitude':'median'
}

null_values_inputation_random_observation = {
    'income':'median', 
    'requested_amount':'median', 
    'requested_period':'median', 
    'financial_obligations':'median', 
    'offer_amount':'observation', 
    'offer_period':'observation',
    'interest_rate':'observation', 
    'fee':'observation', 
    'offer_monthly_obligation':'observation',
    'latitude':'median',
    'longitude':'median'
}

In [12]:
# funkcja generująca wartości z rozkładu normalnego o zadanych parametrach
def generate_from_normal_dist(sample_length, mean, std):
    result = []
    for i in range(0, sample_length):
        out = np.random.normal(mean, 0.333 * std)
        out = max(out, mean - std)
        out = min(out, mean + std)
        result.append(out)
    return result


# funkcja uzupełniająca serię danych wg zadanego sposobu uzupełnienia
def refill_nulls_in_series(series, inputation_parameter):
    features_list = list(series)
    output_list = []
    if type(inputation_parameter) is list:
        counter = 0
        for item in features_list:
            if np.isnan(item):
                output_list.append(inputation_parameter[counter])
                counter += 1
            else:
                output_list.append(item) 
    else:
        output_list = [inputation_parameter if np.isnan(x) else x for x in features_list]
    return pd.Series(output_list)


# funkcja uzupełniająca dataset zmienna po zmiennej - zwraca dataset z uzupełnionymi brakami
def refill_nulls(dataset, dataset_desc, options_dict):
    dataset_size = dataset.shape[0]
    nulls_to_cover = dataset_desc[dataset_desc['count'] < dataset_size]
    refilled_dataset = dataset.copy()
    for feature in nulls_to_cover.index:
        inputation_method = options_dict[feature]
        if inputation_method is 'distribution':
            nulls_number = int(dataset_size - dataset_desc['count'][feature])
            mean = dataset_desc['mean'][feature]
            std = dataset_desc['std'][feature]
            inputation_parameter = generate_from_normal_dist(nulls_number, mean, std) 
        elif inputation_method in {'mean', 'median'}:
            inputation_parameter = dataset_desc[inputation_method][feature]
        else:
            inputation_parameter = None
        if inputation_parameter is not None:
            refilled_dataset[feature] = refill_nulls_in_series(refilled_dataset[feature], inputation_parameter)
    return refilled_dataset

In [13]:
# wariant 1 - wszystkie braki uzupełnione medianami

dataset_inputation_median = refill_nulls(dataset, numeric_features_stats, null_values_inputation_median) 
numeric_features_stats_median = dataset_inputation_median[numeric].describe(percentiles=[.01, .25, .5, .75, .99]).transpose().rename(columns={'50%':'median'})
numeric_features_stats_median.style.format(columns_formats)

,count,mean,std,min,1%,25%,median,75%,99%,max
income,87020,34876.94,35569.25,2583.00,2583.00,16500.00,25000.00,40000.00,250000.00,250000.00
requested_amount,87020,221574.00,287643.57,0.00,0.00,0.00,100000.00,300000.00,1500000.00,1500000.00
requested_period,87020,2.13,2.01,0.00,0.00,0.00,2.00,4.00,5.00,5.00
financial_obligations,87020,3258.54,7192.56,0.00,0.00,0.00,0.00,3500.00,40500.00,40552.00
offer_amount,87020,355620.66,233211.03,60000.00,80000.00,300000.00,300000.00,360000.00,1500000.00,1500000.00
offer_period,87020,3.93,0.91,1.00,1.00,4.00,4.00,4.00,5.00,5.00
interest_rate,87020,18.38,3.34,12.99,13.00,18.00,18.00,18.00,33.00,37.00
fee,87020,4335.77,2502.19,700.00,1000.00,4000.00,4000.00,4000.00,17000.00,24000.00
offer_monthly_obligation,87020,9882.17,4051.02,2405.32,2821.24,9392.97,9392.97,9392.97,29916.10,40614.48
latitude,87020,20.13,6.01,8.08,9.91,13.08,18.98,26.45,30.75,34.55


In [14]:
# wariant 2 - braki zmiennych finansowych uzupełnione przez losowanie z rozkładu N

dataset_inputation_distribution = refill_nulls(dataset, numeric_features_stats, null_values_inputation_distribution) 
numeric_features_stats_distribution = dataset_inputation_median[numeric].describe(percentiles=[.01, .25, .5, .75, .99]).transpose().rename(columns={'50%':'median'})
numeric_features_stats_distribution.style.format(columns_formats)

,count,mean,std,min,1%,25%,median,75%,99%,max
income,87020,34876.94,35569.25,2583.00,2583.00,16500.00,25000.00,40000.00,250000.00,250000.00
requested_amount,87020,221574.00,287643.57,0.00,0.00,0.00,100000.00,300000.00,1500000.00,1500000.00
requested_period,87020,2.13,2.01,0.00,0.00,0.00,2.00,4.00,5.00,5.00
financial_obligations,87020,3258.54,7192.56,0.00,0.00,0.00,0.00,3500.00,40500.00,40552.00
offer_amount,87020,355620.66,233211.03,60000.00,80000.00,300000.00,300000.00,360000.00,1500000.00,1500000.00
offer_period,87020,3.93,0.91,1.00,1.00,4.00,4.00,4.00,5.00,5.00
interest_rate,87020,18.38,3.34,12.99,13.00,18.00,18.00,18.00,33.00,37.00
fee,87020,4335.77,2502.19,700.00,1000.00,4000.00,4000.00,4000.00,17000.00,24000.00
offer_monthly_obligation,87020,9882.17,4051.02,2405.32,2821.24,9392.97,9392.97,9392.97,29916.10,40614.48
latitude,87020,20.13,6.01,8.08,9.91,13.08,18.98,26.45,30.75,34.55


In [15]:
# wariant 3 - braki zmiennych finansowych uzupełnione danymi z datasetu z innych rekordów 

dataset_inputation_random_observation = refill_nulls(dataset, numeric_features_stats, null_values_inputation_random_observation) 
observations_set = dataset_inputation_random_observation[~np.isnan(dataset['fee'])].reset_index(drop=True)
observations_set = observations_set[['offer_amount', 'offer_period', 'interest_rate', 'fee', 'offer_monthly_obligation']]
observations_indexes_list = [x for x in range(0, observations_set.shape[0])]

def data_change(row):
    if np.isnan(row['offer_amount']):
        random = observations_set.loc[sample(observations_indexes_list, 1)[0]]
        row['offer_amount'] = random['offer_amount']
        row['offer_period'] = random['offer_period']
        row['interest_rate'] = random['interest_rate']
        row['fee'] = random['fee']
        row['offer_monthly_obligation'] = random['offer_monthly_obligation']
    elif np.isnan(row['interest_rate']):
        random = observations_set.loc[sample(observations_indexes_list, 1)[0]]
        row['interest_rate'] = random['interest_rate']
        row['fee'] = random['fee']
        row['offer_monthly_obligation'] = random['offer_monthly_obligation']
    elif np.isnan(row['fee']):
        random = observations_set.loc[sample(observations_indexes_list, 1)[0]]
        row['fee'] = random['fee']
    return row

dataset_inputation_random_observation = dataset_inputation_random_observation.apply(lambda x: data_change(x), axis=1 )
numeric_features_stats_random_observation = dataset_inputation_random_observation[numeric].describe(percentiles=[.01, .25, .5, .75, .99]).transpose().rename(columns={'50%':'median'})
numeric_features_stats_random_observation.style.format(columns_formats)

,count,mean,std,min,1%,25%,median,75%,99%,max
income,87020,34876.94,35569.25,2583.00,2583.00,16500.00,25000.00,40000.00,250000.00,250000.00
requested_amount,87020,221574.00,287643.57,0.00,0.00,0.00,100000.00,300000.00,1500000.00,1500000.00
requested_period,87020,2.13,2.01,0.00,0.00,0.00,2.00,4.00,5.00,5.00
financial_obligations,87020,3258.54,7192.56,0.00,0.00,0.00,0.00,3500.00,40500.00,40552.00
offer_amount,87020,381952.54,289237.02,60000.00,60000.00,200000.00,300000.00,500000.00,1500000.00,1500000.00
offer_period,87020,3.89,1.16,1.00,1.00,3.00,4.00,5.00,5.00,5.00
interest_rate,87020,19.23,5.85,12.99,12.99,15.25,18.00,20.00,37.00,37.00
fee,87020,5069.44,4365.49,700.00,750.00,2000.00,4000.00,6250.00,24000.00,24000.00
offer_monthly_obligation,87020,10939.76,7081.27,2405.32,2411.32,6491.28,9425.76,12945.58,40614.48,40614.48
latitude,87020,20.13,6.01,8.08,9.91,13.08,18.98,26.45,30.75,34.55


## Analiza zmiennych kategorycznych

W datasecie występuje 9 zmiennych, które jednoznacznie można zaliczyć jako zmienne kategoryczne.
Są to:

'gender', 'mobile_verification_flag', 'var_5', 'var_1', 'filled_form_flag', 'device', 'var_2', 'source', 'var_4'

Do zmiennych kategorycznych nie zaliczyłem zmiennych 'employer_name' oraz 'account_bank', które podobnie jak zmienna 'city' mają charakter mocno specyficzny dla danego rekordu. Można byłoby podjąć próbę stowrzenia zmiennych numerycznych / kategorycznych opartych na tych informacjach. Ja pominąłem te zmienne w dalszej analizie.

Większość zmiennych kategorycznych nie ma podanej interpretacji, trudno więc ocenić ad-hoc ich użyteczność w wykonywanej analizie.

In [16]:
categorical = ['gender', 'mobile_verification_flag', 'var_5', 'var_1', 
               'filled_form_flag', 'device', 'var_2', 'source', 'var_4']

Dla każdej zmiennej zestawiłem:
- liczbę i nazwy klas
- liczebność obserwacji w danej klasie
- udział procentowy 'disbursed_flag'=1 w danej klasie

In [17]:
categorical_variables_stats = pd.DataFrame()

def categorical_variables_analysis(dataset, feature_name, target_feature):
    selected_feature = dataset[[feature_name, target_feature]].fillna('_None')
    output = selected_feature.groupby(feature_name).mean().sort_values(by=target_feature, ascending=True).reset_index()
    output = output.rename(columns={feature_name:'feature_value', target_feature: 'target feature %'})
    output['feature_name'] = feature_name
    output['feature_code'] = output.index
    feature_value_frequency = selected_feature.groupby(feature_name).count().reset_index().rename(columns={feature_name:'feature_value', target_feature: 'count'})
    output = output.merge(feature_value_frequency, on='feature_value')
    output = output[['feature_name', 'feature_value', 'count', 'target feature %', 'feature_code']]
    return output

for feature in categorical:
    feature_statistics = categorical_variables_analysis(dataset, feature, 'disbursed_flag')
    display(HTML(feature_statistics.to_html()))
    categorical_variables_stats = categorical_variables_stats.append(feature_statistics) 
    

,feature_name,feature_value,count,target feature %,feature_code
0,gender,Female,37172,0.008555,0
1,gender,Male,49848,0.019158,1


,feature_name,feature_value,count,target feature %,feature_code
0,mobile_verification_flag,N,30539,0.007433,0
1,mobile_verification_flag,Y,56481,0.018520,1


,feature_name,feature_value,count,target feature %,feature_code
0,var_5,1,12236,0.001553,0
1,var_5,2,4485,0.005128,1
2,var_5,0,29087,0.005191,2
3,var_5,3,6759,0.007841,3
4,var_5,4,1815,0.008815,4
5,var_5,11,5204,0.015373,5
6,var_5,8,2515,0.015905,6
7,var_5,6,983,0.018311,7
8,var_5,5,975,0.022564,8
9,var_5,9,2281,0.022797,9


,feature_name,feature_value,count,target feature %,feature_code
0,var_1,HVYS,186,0.000000,0
1,var_1,HCXG,78,0.000000,1
2,var_1,HAXF,15,0.000000,2
3,var_1,HBXB,4479,0.003349,3
4,var_1,HCXD,237,0.008439,4
5,var_1,HCYS,217,0.009217,5
6,var_1,HBXC,9010,0.010211,6
7,var_1,HBXX,59294,0.011502,7
8,var_1,HBXH,970,0.012371,8
9,var_1,HAYT,508,0.013780,9


,feature_name,feature_value,count,target feature %,feature_code
0,filled_form_flag,N,67530,0.012232,0
1,filled_form_flag,Y,19490,0.022935,1


,feature_name,feature_value,count,target feature %,feature_code
0,device,Mobile,22704,0.011628,0
1,device,Web-browser,64316,0.015688,1


,feature_name,feature_value,count,target feature %,feature_code
0,var_2,A,5,0.000000,0
1,var_2,D,634,0.000000,1
2,var_2,F,544,0.011029,2
3,var_2,C,14210,0.012386,3
4,var_2,G,33032,0.013381,4
5,var_2,E,1315,0.015970,5
6,var_2,B,37280,0.016845,6


,feature_name,feature_value,count,target feature %,feature_code
0,source,S140,1,0.000000,0
1,source,S156,308,0.000000,1
2,source,S155,4,0.000000,2
3,source,S154,1,0.000000,3
4,source,S150,10,0.000000,4
5,source,S139,3,0.000000,5
6,source,S138,3,0.000000,6
7,source,S136,3,0.000000,7
8,source,S135,2,0.000000,8
9,source,S162,36,0.000000,9


,feature_name,feature_value,count,target feature %,feature_code
0,var_4,0,2546,0.000393,0
1,var_4,7,2302,0.001303,1
2,var_4,6,232,0.004310,2
3,var_4,1,23906,0.006107,3
4,var_4,3,25260,0.016152,4
5,var_4,4,6577,0.018245,5
6,var_4,5,20266,0.022057,6
7,var_4,2,5931,0.024785,7


Zmienne na ogół charakteryzują się dość równomiernym rozkładem obserwacji w klasach. 
Wyjątek stanowią zmienne 'var_1', 'var_2' i 'source'. 
Dodatkowo, zmienne 'var_1' i 'source' mają sporą liczność klas.

Dla tych zmiennych należałoby zagregować klasy mało liczne z innymi.
Trudno jednak przyjąć rozsądne kryterium agregacji bez znajomości znaczenia tych zmiennych.

Na potrzeby dalszych analiz pogrupowałem wartości słownikowe dla tych zmiennych 'ręcznie', przyjmując jako kryterium agregacji podobny poziom procentowy zmiennej objaśnianej w klasach.

In [19]:
var_1_aggregate = {0:0, 1:0, 2:0, 3:0, 4:1, 5:1, 6:1, 7:2, 8:3, 9:3, 10:3, 11:3, 12:3, 13:4, 14:4, 15:4, 16:5, 17:5, 18:5}
var_2_aggregate = {0:0, 1:0, 2:0, 3:0, 4:1, 5:2, 6:2}
source_aggregate = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0, 11:0, 12:0, 13:0, 14:0, 15:0, 16:1, 17:1, 18:2, 19:2, 20:2, 21:2, 22:2, 23:2, 24:3, 25:4, 26:4, 27:5, 28:5, 29:5}

categorical_variables_stats['feature_code'] = categorical_variables_stats.apply(lambda x: var_1_aggregate[x['feature_code']] if x['feature_name']=='var_1' else x['feature_code'], axis=1)
categorical_variables_stats['feature_code'] = categorical_variables_stats.apply(lambda x: var_2_aggregate[x['feature_code']] if x['feature_name']=='var_2' else x['feature_code'], axis=1)
categorical_variables_stats['feature_code'] = categorical_variables_stats.apply(lambda x: source_aggregate[x['feature_code']] if x['feature_name']=='source' else x['feature_code'], axis=1)

categorical_variables_stats.to_csv('encoders_.csv', index=False, sep = ';')

Ostatnim krokiem jest kodowanie zmiennych kategorycznych oraz eksport przygotowanych zbiorów danych do pliku

In [20]:
def categorical_variables_encoder(dataset, features_mapper):
    features_for_encoding = list(set(features_mapper.feature_name))
    for feature in features_for_encoding:
        mapper = features_mapper[features_mapper['feature_name']==feature][['feature_value', 'feature_code']]
        mapper = mapper.set_index('feature_value').to_dict()['feature_code']
        dataset[feature] = dataset.apply(lambda x: mapper[x[feature]], axis=1)
                                
categorical_variables_encoder(dataset_inputation_median, categorical_variables_stats)
categorical_variables_encoder(dataset_inputation_random_observation, categorical_variables_stats)
categorical_variables_encoder(dataset_inputation_distribution, categorical_variables_stats)

In [21]:
dataset_inputation_median.to_csv('dataset_inputation_median.csv', index=False, sep=';')
dataset_inputation_random_observation.to_csv('dataset_inputation_random_observation.csv', index=False, sep=';')
dataset_inputation_distribution.to_csv('dataset_inputation_distribution.csv', index=False, sep=';')